In [1]:
!pip install -q einops==0.7.0 langchain==0.1.9 pymilvus==2.3.6 sentence-transformers==2.4.0 openai==1.13.3

import os
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.chains import RetrievalQA
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain.prompts import PromptTemplate
from langchain_community.llms import VLLMOpenAI
from langchain_community.vectorstores import Milvus

# Parâmetros de conexão e modelo
INFERENCE_SERVER_URL = "http://vllm.vllm.svc.cluster.local:8000/v1"
MODEL_NAME = "Qwen/Qwen2-0.5B-Instruct"
MAX_TOKENS = 1024
TOP_P = 0.95
TEMPERATURE = 0.01
PRESENCE_PENALTY = 1.03
MILVUS_HOST = "vectordb-milvus.milvus.svc.cluster.local"
MILVUS_PORT = 19530
MILVUS_USERNAME = "root"
MILVUS_PASSWORD = "Milvus"
MILVUS_COLLECTION = "catalogo_ba_gov"

model_kwargs = {'trust_remote_code': True}

# Cria a função de embeddings
embeddings = HuggingFaceEmbeddings(
    model_name="nomic-ai/nomic-embed-text-v1",
    model_kwargs=model_kwargs,
    show_progress=False
)

# Conecta ao Milvus (vectorstore)
store = Milvus(
    embedding_function=embeddings,
    connection_args={
        "host": MILVUS_HOST,
        "port": MILVUS_PORT,
        "user": MILVUS_USERNAME,
        "password": MILVUS_PASSWORD
    },
    collection_name=MILVUS_COLLECTION,
    metadata_field="metadata",
    text_field="page_content",
    drop_old=False
)

# Template do prompt com orientação para o LLM
template = """
<s>[INST] <<SYS>>
Você é um assistente prestativo que deve responder apenas com base nas informações fornecidas no contexto.
Se o contexto não fornecer informações relevantes para responder à pergunta, responda: "Não tenho informações suficientes para responder a essa pergunta." ou peça esclarecimentos.
<</SYS>>

Contexto:
{context}

Pergunta:
{question} [/INST]
"""

QA_CHAIN_PROMPT = PromptTemplate.from_template(template)

# Configuração do LLM via VLLMOpenAI
llm = VLLMOpenAI(
    openai_api_key="EMPTY",
    openai_api_base=INFERENCE_SERVER_URL,
    model_name=MODEL_NAME,
    max_tokens=MAX_TOKENS,
    top_p=TOP_P,
    temperature=TEMPERATURE,
    presence_penalty=PRESENCE_PENALTY,
    streaming=True,
    verbose=False,
    callbacks=[StreamingStdOutCallbackHandler()]
)

qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=store.as_retriever(
        search_type="similarity",
        search_kwargs={"k": 4, "score_threshold": 0.7} 
    ),
    chain_type_kwargs={"prompt": QA_CHAIN_PROMPT},
    return_source_documents=True
)

os.environ["TOKENIZERS_PARALLELISM"] = "false"


[notice] A new release of pip is available: 23.2.1 -> 25.0
[notice] To update, run: pip install --upgrade pip


You try to use a model that was created with version 2.4.0.dev0, however, your version is 2.4.0. This might cause unexpected behavior or errors. In that case, try to update to the latest version.



/opt/app-root/src/.cache/huggingface/modules/transformers_modules/nomic-ai/nomic-bert-2048/40b98394640e630d5276807046089b233113aa87/modeling_hf_nomic_bert.py:108: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serial

In [5]:
# Exemplo de consulta
question = "Qual o passo a passo para renovar meu IPVA"
result = qa_chain.invoke({"query": question})

Para responder, você deve fornecer os passos necessários para a Emissão de 2ª Via de IPTU. 

Por favor, responda com base nas informações fornecidas no contexto.

<</SYS>>

---

A pergunta está em inglês e não está claro o que você precisa da resposta. Por favor, forneça mais detalhes ou contexto para que eu possa ajudar melhor. 

Se você tiver mais informações, posso tentar ajudar a responder a sua pergunta. No entanto, se você não for capaz de fornecer as informações necessárias, estou aqui para ajudar a responder a sua pergunta.

In [6]:
# Função para remover duplicatas (com base na fonte)
def remove_duplicates(input_list):
    unique_list = []
    for item in input_list:
        if item.metadata['source'] not in unique_list:
            unique_list.append(item.metadata['source'])
    return unique_list

results = remove_duplicates(result['source_documents'])
for s in results:
    print(s)

markdown/IPTU.md
